# Q1

### part a
write down the design matrix X

In [ ]:
# Use R for this question, but do not use the lm() function.
# Suppose (0, 1), (1, 3), (2, 5), (3, 6), (4, 5), (5, 7) are 6 independently observed data points, and a simple regression model (with intercept) is to be ﬁt to these data.


# Write down the design matrix X.
X = matrix(c(0, 1, 1, 3, 2, 5, 3, 6, 4, 5, 5, 7), nrow = 6, ncol = 2, byrow = TRUE)
head(X)
n = nrow(X)
y <- X[, 2]
X <- X[, 1] # remove y

X1 <- rep(1, n)
X_design_matrix <- cbind(X1, X)
head(X_design_matrix)

### Part b

determine the QR decomposition for X

In [ ]:
# Determine the QR decomposition for X.
QR <- qr(X_design_matrix)
# QR

Q <- qr.Q(QR, complete = TRUE)
R <- qr.R(QR)
Q
R

### part c

Calculate inverse U

In [ ]:
p <- 2  # number of columns in X -> 2
Q1 <- Q[, 1:p]
Q1
# y <- X[, 1] # 这里改成X的第二列是y
Q1y <- t(Q1)%*%y
U <- qr.R(QR)
inverse <- solve(U)
inverse


### part d 

slope and intercept estimate

In [ ]:
betahat <- solve(U, Q1y)
betahat

### part e

determine the residual sum of square

In [ ]:
Q2 <- Q[, -(1:p)]
Q2
Q2y <- t(Q2)%*%y

SSE <- t(Q2y)%*%Q2y
SSE

### part f

estimate the error variance

In [ ]:
# estimate the error variance
MSE <- SSE/(n-p)
MSE

### part g

calculate the test statistic used to determine whether the slope is 0 or not

In [ ]:
Q2Ty <- QTy[-(1:p)] 
MSE <- mean(Q2Ty^2) # MSE

U_1 <- solve(U) # inverse of U
U.diag <- diag(U_1%*%t(U_1))# extracts the diagonal from a matrix
T <- betahat/sqrt(MSE * U.diag) 
T[2] # slope of the 2nd value

### part h

In [ ]:
# conﬁdence interval
Cii <- sqrt(diag(solve(t(U)%*%U))) 
SEii <- Cii*as.numeric(sqrt(MSE))

In [ ]:
# A 95% conﬁdence interval for the slope is
betahat[2] + SEii[2]*qt(c(.025, .975), df = n-p)

**Answer**

Based on the 95% CI, we can tell that there are 95% possibility that the slope is lying in the range of (0.39820328100234, 1.71608243328337). 

It CI does not contain the 0, which means we can reject the nul hypothesis(slope = 0).

# Q2

The p13.2 data frame in the MPV package has 20 observations on home ownership as it relates to family income. Fit a logistic regression model to the data and use the output to

In [ ]:
library(MPV)
head(p13.2)
# X: family income
# Y: home ownership

In [ ]:
nrow(p13.2)

### part a

identify the logit of the probability of home ownership as a linear function of family income.

In [ ]:
p13.2.glm <- glm(y~x, data = p13.2, family = binomial)
coef(p13.2.glm)

### part b

determine if the logistic model is reasonable.

In [ ]:
min(p13.2$x)
max(p13.2$x)

In [ ]:
plot(y~x, data = p13.2, col=ifelse(p13.2$y==1, "red", "blue"), xlab="Family Income", ylab="Home Ownership", pch=20, cex=1.5)
xpoints <- seq(38000, 54000, length=401)
ypredict <- predict(p13.2.glm, newdata=data.frame(x = xpoints), type="response")
lines(xpoints, ypredict)

The curve seems to predict low probability of owning a home at low values of income and higher probability at high values of income.

### part c

estimate the probability that a family with an income of $40000 owns their home.

In [ ]:
y_pred <- predict(p13.2.glm, newdata=data.frame(x = 40000), type="response")
cat("There is a", y_pred*100, "% chance that a family with an income of $40,000 will own a home.")

# Q3

Inspect the dependency of the number of seizures ( y ) in the age of the patient (age) and the treatment (trt).

In [ ]:
# ? MASS::epil
library(MASS)

In [43]:
# X: the age of the patient (age) and the treatment (trt).
# Y : number of seizures
head(epil)
df <- data.frame(age = epil$age, trt = epil$trt, y = epil$y)
head(df)

,y,trt,base,age,V4,subject,period,lbase,lage
,<int>,<fct>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
1,5,placebo,11,31,0,1,1,-0.7563538,0.11420370
2,3,placebo,11,31,0,1,2,-0.7563538,0.11420370
3,3,placebo,11,31,0,1,3,-0.7563538,0.11420370
4,3,placebo,11,31,1,1,4,-0.7563538,0.11420370
5,3,placebo,11,30,0,2,1,-0.7563538,0.08141387
6,5,placebo,11,30,0,2,2,-0.7563538,0.08141387


,age,trt,y
,<int>,<fct>,<int>
1,31,placebo,5
2,31,placebo,3
3,31,placebo,3
4,31,placebo,3
5,30,placebo,3
6,30,placebo,5


### part a

Fit a Poisson regression with glm.

In [47]:
# find how many unique value in trt
unique(df$trt)
# convert the trt into 1 and 0
df$trt <- ifelse(df$trt == "placebo", 0, 1)
head(df)

[1] placebo   progabide
Levels: placebo progabide

,age,trt,y
,<int>,<dbl>,<int>
1,31,0,5
2,31,0,3
3,31,0,3
4,31,0,3
5,30,0,3
6,30,0,5


In [51]:
# x contain age and trt
df.glm <- glm(y~age+trt, data=df)
coef(df.glm)

(Intercept)         age         trt 
 11.7134962  -0.1080393  -0.7566001

### part b

Are the coeﬃcients signiﬁcant?

In [52]:
summary(df.glm)


Call:
glm(formula = y ~ age + trt, data = df)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-9.445  -5.830  -3.500   0.015  93.420  

Coefficients:
            Estimate Std. Error t value Pr(>|t|)   
(Intercept)  11.7135     3.9364   2.976  0.00323 **
age          -0.1080     0.1296  -0.834  0.40538   
trt          -0.7566     1.6217  -0.467  0.64126   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for gaussian family taken to be 153.2001)

    Null deviance: 35825  on 235  degrees of freedom
Residual deviance: 35696  on 233  degrees of freedom
AIC: 1862.2

Number of Fisher Scoring iterations: 2


Based on the P_value from Pr(>|t|), both "age" and "trt" are not significant.

Only intercept has a significant value.

### Part c

What is the 95% conﬁdence interval for the estimates of the coeﬃcients.

In [53]:
# find 95% confidence interval

t_test <- function(x, y, alpha = 0.05) {
      n1 <- length(x)
      n2 <- length(y)
      xbar <- mean(x)
      ybar <- mean(y)
      s1 <- sd(x)
      s2 <- sd(y)
      se <- sqrt(s1^2/n1 + s2^2/n2)
      t <- (xbar - ybar)/se
      df <- n1 + n2 - 2
      p <- 2*pt(-abs(t), df)
      if (p < alpha) {
            cat("Reject the null hypothesis that the means are equal.")
      } 
      else {
            cat("Fail to reject the null hypothesis that the means are equal.")
      }
}
t_test(df$age[df$trt == 0], df$age[df$trt == 1])

Fail to reject the null hypothesis that the means are equal.